In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import fashion_mnist


In [31]:
x_train = pd.read_csv('train.csv')
y_train = np.asarray(x_train['label'], dtype='float32')
x_train = x_train.drop('label', axis=1)
x_test = pd.read_csv('test.csv')

In [32]:
x_train.shape

(42000, 784)

In [34]:
x_train = x_train.values.reshape(-1,28,28,1)
x_train = x_train.astype('float32') / 255.

In [39]:
x_test = x_test.values.reshape(-1,28,28,1)
x_test = x_test.astype('float32') / 255.

In [40]:
#Adding noise to images
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape)
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape)

x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

In [6]:
class Denoise(Model):
    def __init__(self):
        super(Denoise, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(28,28,1)),
            layers.Conv2D(16, (3,3), activation='relu', padding='same', strides=2),
            layers.Conv2D(8, (3,3), activation='relu', padding='same', strides=2)
        ])
        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same')
        ])
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
autoencoder = Denoise()

In [7]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [38]:
autoencoder.fit(x_train_noisy, x_train, epochs=10)

Train on 42000 samples
Epoch 1/10
42000/42000 [==============================] - 22s 535us/sample - loss: 0.0129
Epoch 2/10
42000/42000 [==============================] - 22s 521us/sample - loss: 0.0050
Epoch 3/10
42000/42000 [==============================] - 22s 523us/sample - loss: 0.0046
Epoch 4/10
42000/42000 [==============================] - 22s 520us/sample - loss: 0.0045
Epoch 5/10
42000/42000 [==============================] - 22s 514us/sample - loss: 0.0044
Epoch 6/10
42000/42000 [==============================] - 22s 521us/sample - loss: 0.0043
Epoch 7/10
42000/42000 [==============================] - 22s 516us/sample - loss: 0.0043
Epoch 8/10
42000/42000 [==============================] - 22s 519us/sample - loss: 0.0043
Epoch 9/10
42000/42000 [==============================] - 22s 525us/sample - loss: 0.0042
Epoch 10/10
42000/42000 [==============================] - 22s 521us/sample - loss: 0.0042
